# D - classification of change dates
### REQUIREMENT: (B) and (C) to be run prior (D)
### This program carries out the classification of the changement detected by program (B) using the following function:
#### dates_change_classification(dteDebut, dteFin, dteChgEnd, dteExe):

In [ ]:
import os, sys, datetime, pandas as pd
sys.path.append("/home/gswinnen/SARSAR_Package_RenPri/code/") # emplacement des modules RenPri

#sys.path.append("/home/issep/sarsar-issep/SARSAR_utils/")                   # emplacement des modules RenPri
#sys.path.append("/home/issep/sarsar-issep/SARSAR_utils/rme_chg_detection_module/") # emplacement de la fonction de Mattia

from issep import sarsar_admin
from os.path import join
from lecture_ini import config

In [ ]:
# Defines a function to convert formatted strings into date objects; on which intervals can be calculated.
dateparse = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d')

In [ ]:
def dates_change_classification(dteDebut, dteFin, dteChgEnd, dteExe):

    dates = config(section='dates')
    nbrDays_S1 = int(dates['days_s1'])
    nbrDays_S2 = int(dates['days_s2'])
    
    # Define Database connection parameters
    # NOTE: password is in ~/.pgpass
    credentials = config(section='postgresql')

    db_credentials = {
        'host': credentials['host'],
        'user': credentials['user'],
        'db' : credentials['database']
    }

    # ALWAYS prepare env et the beginning
    print('> Preparing env (DB credentials, etc)')
    sarsar_admin.prepare_env(db_credentials)

    # Ouvre la connexion à la DB
    conn = sarsar_admin._create_or_get_db_connection()
    cur = None

    # Liste de dictionnaires 'sar_id_segment'
    sar_id_segments = []

    try:
        import psycopg2.extras
        cur = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)
        cur2 = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)

        ## Creation de la table résultat, lecture des dates de changements, 
        ## calculs des amplitudes dans les indices lorsqu'il y a une date de changement

        table_name = 'classif_bimestrial_chg_dates'

        # strSQL = 'DROP TABLE IF EXISTS {0};'.format(table_name)
        # cur.execute(strSQL)
        # conn.commit()

        strSQL = 'CREATE TABLE IF NOT EXISTS "{0}" (ID_Segment TEXT, dteExe DATE, dteChgEnd DATE, date DATE, NDVI_amplitude NUMERIC(4,3), BAI_amplitude NUMERIC(4,3), VH_amplitude NUMERIC(4,3), vegetation INTEGER, soil INTEGER, building INTEGER, nImages_a INTEGER, nImages_p INTEGER);'.format(table_name)
        cur.execute(strSQL)

        # Liste les sar_id_segment pour lesquels j'ai des observations dans l'intervalle de dates
        strSQL = '''SELECT DISTINCT sar_id_segment 
                    FROM sar_index_stats WHERE index_name NOT IN ('BI2','VV','SBI','NDVI','BI2_part1','BI','BAI','VH') 
                    AND substring(index_name,1,2) != 'VV' AND acq_date BETWEEN '{0}' AND '{1}' 
                    ORDER BY sar_id_segment;'''.format(dteDebut, dteFin)
        cur.execute(strSQL)
        sites = [item[0] for item in cur.fetchall()]

        # DEBUG: force la liste
    #        sites = ['62003-ISA-0007-01', '62063-ISA-0073-01', '62096-ISA-0056-01','52012-ISA-0010-01']

    #    i_debug = 0               # DEBUG
    #    sites = sites[i_debug:]   # DEBUG

        # Cherche les dates de changements détectées par 02_new (réunit le travail de 02 et 03 dans postgres)
        for site in sites:
    #        i_debug += 1
    #        print(site, i_debug)  # DEBUG

            strSQL = 'SELECT * FROM "{0}" ORDER BY change_date;'.format('{0}_dates_{1}'.format(site, dteExe))
            cur.execute(strSQL)

            # S'il y a au moins un changement détecté
            if cur.rowcount > 0:

                # Converti le fetchall en liste
                dates = [item[0] for item in cur.fetchall()]  # .strftime("%Y-%m-%d")

                # Converti la liste en dataframe
                listeDates = pd.DataFrame (dates, columns = ['Change date'])

    #            print(listeDates)  # DEBUG

                # Ajoute au dataframe des colonnes calculées
                listeDates['a_debut_S2'] = listeDates['Change date']
                listeDates['a_fin_S2'] = listeDates['Change date'] + datetime.timedelta(days = nbrDays_S2)
                listeDates['p_debut_S2'] = listeDates['a_debut_S2'] - datetime.timedelta(days = 365)
                listeDates['p_fin_S2'] = listeDates['a_fin_S2'] - datetime.timedelta(days = 365)

                listeDates['a_debut_S1'] = listeDates['Change date']
                listeDates['a_fin_S1'] = listeDates['Change date'] + datetime.timedelta(days = nbrDays_S1)
                listeDates['p_debut_S1'] = listeDates['a_debut_S1'] - datetime.timedelta(days = 365)
                listeDates['p_fin_S1'] = listeDates['a_fin_S1'] - datetime.timedelta(days = 365)

                # For each date of change, check that we have the smoothed data for each indice; 
                # and calculate the difference for the same period compared to the previous year.
                for i in listeDates.index:
                    NDVI_amplitude = None
                    BAI_amplitude = None
                    VH_amplitude = None
                    vegetation = None
                    soil = None
                    building = None

    # NDVI__________________
                    # Va rechercher les indices NDVI "lissés" dans la DB
                    table_smooth = '{0}_NDVI_{1}_smoothed'.format(site, dteExe)

                    # Vérifie si on a pour ce site/indice une table "smoothed" contenant des valeurs pour notre intervalle de dates
                    strSQL = "SELECT EXISTS (SELECT * FROM information_schema.tables WHERE table_name = '{0}');".format(table_smooth)
                    cur2.execute(strSQL)

                    # Si on a des NDVI "smoothés", on en demande la valeur moyenne
                    NDVI_EXISTS = cur2.fetchone()[0]

                    if NDVI_EXISTS == True:
                        strSQL = 'SELECT a.moyenne, p.moyenne as moyenne_p, ROUND(a.moyenne-p.moyenne, 3) AS moyenne_chg FROM (SELECT avg(indice) as moyenne FROM "{0}" WHERE dte BETWEEN \'{1}\' AND \'{2}\') a, (SELECT avg(indice) as moyenne FROM "{0}" WHERE dte BETWEEN \'{3}\' AND \'{4}\') p;'.format(table_smooth, listeDates['a_debut_S2'][i].strftime('%Y-%m-%d'), listeDates['a_fin_S2'][i].strftime('%Y-%m-%d'), listeDates['p_debut_S2'][i].strftime('%Y-%m-%d'), listeDates['p_fin_S2'][i].strftime('%Y-%m-%d'))
                        cur2.execute(strSQL)
                        NDVI_amplitude = cur2.fetchone()['moyenne_chg']

    # BAI__________________
                    # Va rechercher les indices BAI "lissés" dans la DB
                    table_smooth = '{0}_BAI_{1}_smoothed'.format(site, dteExe)

#                    print('table_smooth =', table_smooth)

                    # Vérifie si on a pour ce site/indice une table "smoothed" contenant des valeurs pour notre intervalle de dates
                    strSQL = "SELECT EXISTS (SELECT * FROM information_schema.tables WHERE table_name = '{0}');".format(table_smooth)
                    cur2.execute(strSQL)

                    # Si on a des BAI "smoothés", on en demande la valeur moyenne
                    BAI_EXISTS = cur2.fetchone()[0]

                    if BAI_EXISTS == True:
                        strSQL = 'SELECT a.moyenne, p.moyenne as moyenne_p, ROUND(a.moyenne-p.moyenne, 3) AS moyenne_chg FROM (SELECT avg(indice) as moyenne FROM "{0}" WHERE dte BETWEEN \'{1}\' AND \'{2}\') a, (SELECT avg(indice) as moyenne FROM "{0}" WHERE dte BETWEEN \'{3}\' AND \'{4}\') p;'.format(table_smooth, listeDates['a_debut_S2'][i].strftime('%Y-%m-%d'), listeDates['a_fin_S2'][i].strftime('%Y-%m-%d'), listeDates['p_debut_S2'][i].strftime('%Y-%m-%d'), listeDates['p_fin_S2'][i].strftime('%Y-%m-%d'))
                        print(strSQL)
                        cur2.execute(strSQL)
                        BAI_amplitude = cur2.fetchone()['moyenne_chg']

    # VH__________________
                    # Va rechercher les indices NDVI "lissés" dans la DB
                    table_smooth = '{0}_VH_{1}_smoothed'.format(site, dteExe)

                    # Vérifie si on a pour ce site/indice une table "smoothed" contenant des valeurs pour notre intervalle de dates
                    strSQL = "SELECT EXISTS (SELECT * FROM information_schema.tables WHERE table_name = '{0}');".format(table_smooth)
                    cur2.execute(strSQL)

                    # Si on a des VH "smoothés", on en demande la valeur moyenne
                    VH_EXISTS = cur2.fetchone()[0]

                    if VH_EXISTS == True:
                        strSQL = 'SELECT a.moyenne, p.moyenne as moyenne_p, ROUND(a.moyenne-p.moyenne, 3) AS moyenne_chg FROM (SELECT avg(indice) as moyenne FROM "{0}" WHERE dte BETWEEN \'{1}\' AND \'{2}\') a, (SELECT avg(indice) as moyenne FROM "{0}" WHERE dte BETWEEN \'{3}\' AND \'{4}\') p;'.format(table_smooth, listeDates['a_debut_S1'][i].strftime('%Y-%m-%d'), listeDates['a_fin_S1'][i].strftime('%Y-%m-%d'), listeDates['p_debut_S1'][i].strftime('%Y-%m-%d'), listeDates['p_fin_S1'][i].strftime('%Y-%m-%d'))
                        cur2.execute(strSQL)
                        VH_amplitude = cur2.fetchone()['moyenne_chg']


    # Vérifie le nombre d'images réelles sur lesquelles la détection s'est opérée (représentativité...)
                    nImages_a = 0
                    nImages_p = 0

                    strSQL = 'SELECT count(*) FROM sar_index_stats WHERE sar_id_segment = \'{0}\' AND index_name = \'NDVI\' AND acq_date BETWEEN \'{1}\' AND \'{2}\';'.format(site, listeDates['a_debut_S2'][i].strftime('%Y-%m-%d'), listeDates['a_fin_S2'][i].strftime('%Y-%m-%d'))
                    cur2.execute(strSQL)
                    nImages_a = cur2.fetchone()[0]

                    strSQL = 'SELECT count(*) FROM sar_index_stats WHERE sar_id_segment = \'{0}\' AND index_name = \'NDVI\' AND acq_date BETWEEN \'{1}\' AND \'{2}\';'.format(site, listeDates['p_debut_S2'][i].strftime('%Y-%m-%d'), listeDates['p_fin_S2'][i].strftime('%Y-%m-%d'))
                    cur2.execute(strSQL)
                    nImages_p = cur2.fetchone()[0]


    # Qualification des changementd__________

                    # changements de l'indice NDVI
                    if NDVI_amplitude != None:
                        if NDVI_amplitude >= 0.1:
                            vegetation = 2  # « Augmentation végétation »

                        elif NDVI_amplitude <= -0.1:
                            vegetation = 3  # « Diminution végétation »

                        else:
                            vegetation = 0  # « Pas de changement végétation »


                    # changements de l'indice BAI
                    if BAI_amplitude != None:

                        if abs(BAI_amplitude) >= 0.05:
                            soil = 1  # « Changement de sol »

                        else:
                            soil = 0  # « Pas de changement sol »


                    # changements de l'indice VH
                    if VH_amplitude != None:
                        if VH_amplitude >= 0.135:
                            building = 2  # « Augmentation bâtiment »

                        elif VH_amplitude <= -0.135:
                            building = 3  # « Diminution bâtiment »

                        else:
                            building = 0  # « Pas de changement bâtiment »

                    strSQL = '''INSERT INTO {0} (ID_Segment, dteExe, dteChgEnd, date, NDVI_amplitude, BAI_amplitude, VH_amplitude, vegetation, soil, building, nImages_a, nImages_p) 
                                VALUES (\'{1}\', \'{2}\', \'{3}\', \'{4}\', {5}, {6}, {7}, {8}, {9}, {10}, \'{11}\', \'{12}\');'''.format(table_name, site, f'{dteExe[0:4]}-{dteExe[4:6]}-{dteExe[6:]}', dteChgEnd, listeDates['Change date'][i].strftime('%Y-%m-%d'), NDVI_amplitude, BAI_amplitude, VH_amplitude, vegetation, soil, building, nImages_a, nImages_p)
    #                print(strSQL)
                    cur2.execute(strSQL)

                    # Poste toutes les modifications
                    conn.commit()

        cur2.close()
        cur.close()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
    finally:
        if cur2 is not None:
            cur2.close()

        if cur is not None:
            cur.close()
            
    # ALWAYS release env at the end
    print('> Releasing env')
    sarsar_admin.release_env()